In [1]:
import os
import json
import pandas as pd
import traceback

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
key=os.getenv("OPENAI_API_KEY")

In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,   
    api_key=key 
)

In [10]:
llm.invoke("tell me about django").content

'Django is a high-level, open-source web framework written in Python that encourages rapid development and clean, pragmatic design. It follows the "batteries-included" philosophy, meaning it comes with many built-in features to handle common web development tasks. Django was originally developed to meet the fast-paced requirements of a newsroom and was released publicly in 2005.\n\nHere are some key features and aspects of Django:\n\n### Key Features\n\n1. **ORM (Object-Relational Mapping)**:\n   - Django\'s ORM allows developers to interact with the database using Python code instead of SQL queries. It supports multiple database backends like PostgreSQL, MySQL, SQLite, and Oracle.\n\n2. **URL Routing**:\n   - Django uses a URL dispatcher to map URL patterns to views, enabling clean and manageable URLs.\n\n3. **Templating Engine**:\n   - The Django templating engine helps in creating dynamic web pages by using a combination of HTML and Django Template Language.\n\n4. **Admin Interface*